In [1]:
import os
import time
from contextlib import contextmanager
from redis import Redis
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex
from redisvl.query import VectorQuery

import numpy as np

In [2]:
@contextmanager
def timer(operation_name="Operation"):
    """Context manager for timing operations."""
    start_time = time.perf_counter()
    yield
    end_time = time.perf_counter()
    print(f"{operation_name} took: {end_time - start_time:.4f} seconds")


def generate_fake_embeddings(num_embeddings=10, embedding_dim=128, type=np.float32 ,seed=None):
    """
    Generate fake embeddings using random numbers.

    Args:
        num_embeddings (int): Number of embeddings to generate.
        embedding_dim (int): Dimension of each embedding vector.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        np.ndarray: A matrix of shape (num_embeddings, embedding_dim) with fake embeddings.
    """
    if seed is not None:
        np.random.seed(seed)

    embeddings = np.random.rand(num_embeddings, embedding_dim).astype(np.float32)
    return embeddings


In [3]:
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")  # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

In [4]:
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
client = Redis.from_url(REDIS_URL)
print(client.ping())

True


In [5]:
  # Define the index name
data_size = 100
dimension = 960 
datatype = "float32" 

index_name = "redisflat"
algorithm = "flat" 
#algorithm = "hnsw" 
distance_metric = "cosine" 



# define the scheama
flat_schema = IndexSchema.from_dict(
    {
        "index": {"name": index_name, "prefix": index_name, "storage_type": "hash"},
        "fields": [
            {
                "name": "vector",
                "type": "vector",
                "attrs": {
                    "dims": dimension,
                    "distance_metric": distance_metric,
                    "algorithm": "flat",
                    "datatype": datatype,
                },
            },
            {
                "name": "id",
                "type": "text",
            },
        ],
    }
)

In [6]:
with timer("Index creation"):
    flatindex = SearchIndex(flat_schema, client,validate_on_load=True)
    flatindex.create(overwrite=True, drop=False )

Index creation took: 0.0028 seconds


In [7]:
with timer("Generating fake embeddings"):
    fake_embeddings = generate_fake_embeddings(
        num_embeddings=data_size, embedding_dim=dimension, type=type ,seed=42
    )
    print("Fake embeddings generated.")

Fake embeddings generated.
Generating fake embeddings took: 1.1554 seconds


In [8]:
# data with other fields
#data = [{"id": i, "vector": e.tobytes()} for i, e in enumerate(fake_embeddings)]
#print("Data prepared for loading into index.")
#index.load(data, id_field="id")
#print("Data loaded into index.")

with timer("Data preparation"):
    data = [{"id": "document:" +str(i),"vector": e.tobytes()} for i, e in enumerate(fake_embeddings)]
    #data = [{"vector": e.tobytes()} for i, e in enumerate(fake_embeddings)]
    print("Data prepared for loading into index.")

with timer("Data loading into index"):
    flatindex.load(data)
    print("Data loaded into index.")

Data prepared for loading into index.
Data preparation took: 0.0001 seconds
Data loaded into index.
Data loading into index took: 0.0504 seconds


In [9]:
#lets query redis
with timer("Vector query execution"):
    query = VectorQuery(
        vector=fake_embeddings[0],
        vector_field_name="vector",
        num_results=3,
        return_fields=["vector"],
        return_score=False,
    )
    results = flatindex.query(query)
    print(results)
    print("Query executed.")
    print("Results:", len( results))

[{'id': 'redisflat:01K6F8BCNH9863VDJ70KMJPMW3', 'vector': 'ÿ>\x03bs?c;?A\x19?Z\x1f>\x07\x1f>\x15m=]?\x19??D5?Р<\x03Lx?\x1aU?moY>S0:>d;>ś>pV\x06?\'>\x1b>d\x1c?{\x0e>>ȓ>\x01>K\x01I?IwL>ޤ\x03?{\x17?B>=\x0f\x1b?.>9=*r?3w?!N?V>Q\x08=)/?[>/=ԇ>\x0c=9h?~>\x0f)?>.#\x05?4\x0b?}J=>6x?\x1boF?\x01p?h\x13e?\x0f\x19?k?;=H>@@9=>\x00>C>$(T?_>֏>!\n?qN\x10>\\M?\x02=|?ձE? |K>;\x15P?4?7:?qE?=҈>L=Y\\?w\x1f?wk>*=\x139>n~>x:?6#?` c?(>=66?B?ݯ\x0f?\x1b^E?>\x05?>;<.=\x00="?D>1\x02?GVh?vF\x7f>\x1d>kA?\x11Jj>=MZ>1\x17%>\x00n?N?&"?\x0b\x18_?tM?9\x0c?>~d?U\x12\n?fN?=fe?ZѢ>b=gi>߭>jiQ?X\\?;V\x02?>;pc>{=۬>bq?z>v\x04?\r3?->x?bv?>[>\x0c>֑>\x16\x17=h\x0c\x1c?\x00?bR=>\x1dh?Ou>R_\x14>>S|?Vw>\x13\x11,?B?BWs>cl:?\x12N>!?\x01/"?(\t?=bU?I=>>>\x03\'=D\x17?t-?3<\x18\x03?g>\x0b*%?\x1d2>L0?-\x02>o?J\x0c>@>d=l?P`?\x10>(?y5Q?!\x0e?.\x07? w>=e?́f?\x12"?P>˲>;9?\x05e?\x19\x18c?G?0\\$?Q=%>\x07f?>\x1b?;\x16<P=@)?ۥ;ު$>y\x0c?\x0b 1?&?ۦe>aQ6?mr>ʚ>\x10\x1a??WN&?fY?RY(?|\x11?ؿ=?E>ȇ>ly>8\x17y?\x1bD>*]d?M!?xK?Ӭ\x00?\x13?G+>G>8?e>G2<=%?\\5>p?4t?